# Training a GAN

In this practical session, we will implement a GAN on the MNIST dataset. SOme parts of the code are missing. Complete the code until you are able to generate images. 

In [ ]:
import os
os.environ["KERAS_BACKEND"] = "tensorflow"
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
import keras

from keras.layers import Input
from keras.models import Model, Sequential
from keras.layers.core import Reshape, Dense, Dropout, Flatten
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.normalization import BatchNormalization
from keras.datasets import mnist
from keras.optimizers import Adam
from keras import backend as K
from keras import initializers
import PIL.Image
import IPython.display
import numpy as np


Using TensorFlow backend.


2.2.5


## Data processing
We load the mnist dataset
The 28x28 are flattened. We will consider these images as vectors.

In [ ]:


# The results are a little better when the dimensionality of the random vector is only 10.
# The dimensionality has been left at 100 for consistency with other GAN implementations.
randomDim = 100

# Load MNIST data
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5)/127.5

X_train = X_train.reshape(60000, 784)

11493376/11490434 [==============================] - 1s 0us/step



## Network architectures

Specify the encoder and the decoder architectures. Use a few fully connected layers (Dense). Pay attention to the input and output dimensions 

In [ ]:
# Optimizer
adam = Adam(lr=0.0002, beta_1=0.5)

generator = Sequential()
# ....
#implement a network with 4 FC layers of dimensions 256,512,1024,784 with activations LeakyReLU(0.2),LeakyReLU(0.2),LeakyReLU(0.2),tanh
# ....


generator.summary()


discriminator = Sequential()
# ....
#implement a network with 4 FC layers of dimensions 1024,512,256,1 with activations LeakyReLU(0.2),LeakyReLU(0.2),LeakyReLU(0.2) and "?"
# ....

discriminator.compile(loss='binary_crossentropy', optimizer=adam)

discriminator.summary()

SyntaxError: ignored

In [ ]:
# We combined the generator and discriminator to create a gan model
discriminator.trainable = False
ganInput = Input(shape=(randomDim,))
#....
gan = Model(inputs=ganInput, outputs=ganOutput)
gan.compile(loss='binary_crossentropy', optimizer=adam)

## Display
Some display functions. There is nothing to complete here.

In [ ]:



# Plot the loss from each batch
def plotLoss(epoch):
    plt.figure(figsize=(10, 8))
    plt.plot(dLosses, label='Discriminitive loss')
    plt.plot(gLosses, label='Generative loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.savefig('gan_loss_epoch_%d.png' % epoch)

# Create a wall of generated MNIST images
def plotGeneratedImages(epoch, examples=100, dim=(10, 10), figsize=(10, 10)):
    noise = np.random.normal(0, 1, size=[examples, randomDim])
    generatedImages = generator.predict(noise)
    generatedImages = generatedImages.reshape(examples, 28, 28)

    plt.figure(figsize=figsize)
    for i in range(generatedImages.shape[0]):
        plt.subplot(dim[0], dim[1], i+1)
        plt.imshow(generatedImages[i], interpolation='nearest', cmap='gray')
        plt.axis('off')
    plt.tight_layout()
    plt.savefig('gan_generated_image_epoch_%d.png' % epoch)
    IPython.display.display(IPython.display.Image(data=('gan_generated_image_epoch_%d.png' % epoch)))

# Save the generator and discriminator networks (and weights) for later use


def saveModels(epoch):
    generator.save('gan_generator_epoch_%d.h5' % epoch)
    discriminator.save('gan_discriminator_epoch_%d.h5' % epoch)
    
    


## Main loop

This is now the main loop of the gan. In each loop, we update first the discriminator and then the generator.

Complete the code. Don't forget to specify which networks shoulb be trainable in step of the training loop.

In [ ]:

dLosses = []
gLosses = []


epochs=22
batchSize=128
batchCount = X_train.shape[0] // batchSize
print('Epochs:', epochs)
print('Batch size:', batchSize)
print('Batches per epoch:', batchCount)

for e in range(1, epochs+1):
    print('-'*15, 'Epoch %d' % e, '-'*15)
    for _ in tqdm(range(batchCount)):
        ####### Train discriminator #####
        

        # Get a random set of input noise and images
        # Generate fake MNIST images
        noise = np.random#...
        generatedImages = #...
        
        #we create a batch composed of real and fake images
        imageBatch = X_train[np.random.randint(0, X_train.shape[0], size=batchSize)]
        X = np.concatenate([imageBatch, generatedImages])

        
        # We manually create the labels corresponding to X (1 for real, 0 for generated)
        yDis = #...
        
        
        # Train discriminator to distinguish real and fake
        #...
        dloss = discriminator.train_on_batch(X, yDis)


        ####### Train generator #####
        # Generate fake MNIST images
        noise = np.random #...
        # labels
        yGen = #...
        # Train generator to confuse the discriminator by forcing the discrimitinator to see the generated images as Real(label=1)

        discriminator.trainable = False
        gloss = gan.train_on_batch(noise, yGen)

    # Store loss of most recent batch from this epoch
    dLosses.append(dloss)
    gLosses.append(gloss)

    if e == 1 or e % 1 == 0:
        plotGeneratedImages(e)
        saveModels(e)

    # Plot losses from every epoch
    plotLoss(e)




## DCGAN

Save a copy of your colab. Now, we will implement a version of GAN that uses convolutions: DCGAN. You can find the paper [here](https://arxiv.org/abs/1511.06434). Replace your naive generator and decoder by the DCGAN architectures. We recommend to batch normalization before each leaky-relu activations. This is slower to train but by looking at the images after one epoch, you can already draw an interesting conclusion.